In [1]:
import sys
sys.path.append('../src')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
import tqdm

from xls_r_decoder.wav2phonemes import recognize

/home/boris/Projects/Project-On-Voice-Assistant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
params = {
    'batch_size': 4,
    'phonemes': 392
}

In [4]:
df = pd.read_csv('/home/boris/Projects/МИиМИС/Курсовая/speech_recognition/data/dataset.csv').values.tolist()

train_df, test_df = train_test_split(df, test_size=0.1)
train_df, val_df = train_test_split(train_df)

In [5]:
train_loader = DataLoader(train_df, params["batch_size"], True)
val_loader = DataLoader(val_df, params["batch_size"], True)
test_loader = DataLoader(test_df, params["batch_size"], True)

Let see what is the type of batches

In [6]:
# next(iter(train_loader))

so, we've `ids: list[int], texts: list['str'], normal: list[str], abnormal: list[str]`

In [7]:
abc = "? абвгдеёжзийклмнопрстуфхшщчцьыъэюя"
def vectorize(labels: tuple[str]):
    lengths = torch.LongTensor(size=(len(labels),))
    # letters = torch.zeros(size=(len(labels),  max(map(len, labels)), len(abc)), dtype=float)
    letters = torch.zeros(size=(len(labels),  max(map(len, labels))), dtype=float)

    for i, label in enumerate(labels):
        lengths[i] = len(label)
        j=0
        for c in label.lower():
            if not c in abc:
                lengths[i]-=1
            else:
                # letters[i,j, abc.index(c)]=1
                letters[i,j]= abc.index(c)
                j+=1
    
    return letters, lengths
def decode(X):
    r = []
    for i in range(X.shape[0]):
        s = []
        for j in range(X.shape[1]):
            if X[i,j] > 0:
                s.append(abc[X[i,j]])
        r.append(''.join(s))
    return r

Test of preprocess correctness

In [29]:
class LSTMCorrector(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(params['phonemes']+32, 32)

        self.sigmoid1=nn.Tanh()
        self.fc2=nn.Linear(32, len(abc))
        # self.lstm = nn.RNN(len(abc), len(abc)//4, batch_first=True, num_layers=2)
        # self.fc = nn.Linear(len(abc)//4, len(abc))

    def forward(self, X):
        # X: batch len classes
        hidden = torch.zeros((X.shape[0], X.shape[1], 32))
        for i in range(X.shape[1]):
            k = torch.cat((X[:, i, :], hidden[:,max(0, i-1),:]), 1)
            hidden[:, i, :] = self.fc1(k)
        
        X = self.fc2(hidden)

        X = self.fc1(X)
        X = self.sigmoid1(X)
        # X, _ = self.lstm(X)
        X = self.fc(X)
        X = F.log_softmax(X, dim=-1)
        return X.permute(1,0,2)

In [30]:
model = LSTMCorrector()

In [31]:
loss = nn.CTCLoss(zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [32]:
for epoch in range(1):
    print(f'epoch {epoch}/5')
    epoch_loss = 0.0
    nb=0
    nb_raw = 0
    for b in (pbar := tqdm.tqdm(train_loader, leave=False)):
        labels, lengths = vectorize(b[1])
        phonemes = recognize(b[3])
        prediction = model(phonemes)
        
        ilengths = lengths.clone()
        for i in range(prediction.shape[1]):
            ilengths[i] = prediction.shape[0]
        lv = loss(prediction, labels, ilengths, lengths)

        optimizer.zero_grad()
        lv.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        nb += sum(lengths)
        epoch_loss += lv.item()

        pbar.set_postfix({"loss": epoch_loss/nb})

        nb_raw+=1
        if nb_raw%10==0:
            epoch_loss=0.0
            nb=0
            print(*decode((prediction.argmax(dim=2)).permute(1,0)), sep='\n')

epoch 0/5


  8%|▊         | 10/129 [01:18<15:04,  7.60s/it, loss=tensor(0.3721)]

нннннннннннннннннннннннннннннннннннннннннннкннннннннннннннннкннннннннннннннннннннннннннннкнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннкнннннннннннннньнннннннннннньнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннднннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
ннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннкннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннннкннннннннннннньннннннннннннннннкннннннннннннннннннннннннннннннннннннннннннвнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн


 16%|█▌        | 20/129 [02:42<16:22,  9.01s/it, loss=tensor(0.3347)]

нннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннннннннннннмннннннннннннкннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннньннннннннннннннкннннннннннннннннннннннннкнннннннннннннннмннннннннннннннннннннннннннннннннннннннн
ннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннкннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
ннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннкннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн


...

 39%|███▉      | 50/129 [06:42<08:22,  6.36s/it, loss=tensor(0.4949)]

нннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннньнннннннннннннннннннннннннннннннннннннннннннннкнннннннннннннннкнннннннннннкннннннннннннннннннннннннннннннннннннннн
нннннннннннннннньннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннннннннннннньнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннкннннннннннньннннннннннннннннннннннннннннннннннннннннн
ннннннннннннннннннньнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн


 47%|████▋     | 60/129 [08:15<09:10,  7.97s/it, loss=tensor(0.4478)]

ннннннннннннннннннннннннннннннннньнннннннннннннннннкннннннннннннннннннннннннннньннвннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннннннкнннннннннннннннньннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн
нннннннннннннннннннннннннннннкннннннннннньннннннннннньнннннннннннннднннннддннннннннннннннннннньннннннннннннннннннннннннннннннннннннн
нннннннннннннннннньннннннннннннннкннннннннннннннннннннннннннннннннннньнннннннннннннннннннннннннннннннннннннннннннннннннннннннннннннн


 49%|████▉     | 63/129 [08:42<09:48,  8.92s/it, loss=tensor(0.2749)]

In [14]:
loss(prediction, labels, lengths, lengths)

tensor(nan, grad_fn=<MeanBackward0>)

In [67]:
prediction[:,0,1:]

tensor([[-3.4830, -3.8087, -3.6947,  ..., -3.5795, -3.9501, -3.8384],
        [-4.4772, -4.9877, -4.9001,  ..., -4.6272, -5.3549, -4.9898],
        [-5.4203, -5.9879, -5.9463,  ..., -5.5983, -6.5168, -6.0107],
        ...,
        [-5.9518, -6.5256, -6.5254,  ..., -6.1657, -7.1658, -6.5898],
        [-5.9518, -6.5256, -6.5254,  ..., -6.1657, -7.1658, -6.5898],
        [-5.9518, -6.5256, -6.5254,  ..., -6.1657, -7.1658, -6.5898]],
       grad_fn=<SliceBackward0>)